In [429]:
import pandas as pd
import os

import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn.decomposition import PCA
from sklearn.feature_selection import chi2, SelectKBest
import numpy as np
from sklearn.metrics import accuracy_score


In [430]:
dataframes = dict()
for dataset_name in os.listdir('data'):
    if dataset_name.endswith('.csv'):
        dataframes[dataset_name.split(".")[0]] = pd.read_csv('clean_data/' + dataset_name) 

data = pd.merge(dataframes['teams_post'], dataframes['teams'], on=['tmID', 'year'], how='right')
data = pd.merge(data, dataframes['coaches'], on=['tmID', 'year'], suffixes=('_team', '_coach'))
data = pd.merge(data, dataframes['series_post'], left_on=['tmID', 'year'], right_on=['tmIDWinner', 'year'], suffixes=('_team_post', '_series'), how='left')
data = pd.merge(data, dataframes['series_post'], left_on=['tmID', 'year'], right_on=['tmIDLoser', 'year'], suffixes=('_team_post', '_series'), how='left')
data = pd.merge(data, dataframes['players_teams'], on=['tmID', 'year'], suffixes=('_team', '_player'))
data = pd.merge(data, dataframes['players'], left_on='playerID', right_on='bioID')
data = pd.merge(data, dataframes['awards_players'], on=['playerID'], how='left')

data.shape
label = "playoff"
df = data
df['confID'].value_counts()

df.drop("year_y", axis=1, inplace=True)
df.columns

Index(['year_x', 'tmID', 'W_team_post', 'L_team_post', 'franchID', 'confID',
       'rank', 'playoff', 'firstRound', 'semis',
       ...
       'PostthreeAttempted', 'PostthreeMade', 'PostDQ', 'bioID', 'pos',
       'height', 'weight', 'college', 'birthDate', 'award'],
      dtype='object', length=119)

In [431]:
features = df.columns
toDrop = ["rank","semis","finals","firstRound"]
for col in toDrop:
    features = features.drop(col)

arr = []

for col in df.columns:

    if "post" in col.lower():
        features = features.drop(col)
        arr.append(col)


assert len(features) < len(df.columns)

X = df[features]
df = X

In [432]:
df['playoff']
le = LabelEncoder()
y = le.fit_transform(df['playoff'])
for x in df.columns:
    df[x] = le.fit_transform(df[x])   

/var/folders/97/nk3s9sxx36x77jsk9m_r0xmc0000gn/T/ipykernel_18532/3239734377.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[x] = le.fit_transform(df[x])
/var/folders/97/nk3s9sxx36x77jsk9m_r0xmc0000gn/T/ipykernel_18532/3239734377.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[x] = le.fit_transform(df[x])
/var/folders/97/nk3s9sxx36x77jsk9m_r0xmc0000gn/T/ipykernel_18532/3239734377.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

In [433]:
available_columns = list(features)

key_predictors  = []
features_values = pd.DataFrame(df, columns = available_columns)
target_values   = df["playoff"].values
chi2_results    = chi2(features_values, target_values)
best_chi2_cols  = SelectKBest(chi2, k= 30)

best_chi2_cols.fit(features_values, target_values)

best_chi2_features = features_values.columns[best_chi2_cols.get_support()]

for k in (best_chi2_features):
     key_predictors.append(k)

     
key_predictors.append("playoff")
key_predictors.append("year_x")
key_predictors.append("confID")
df = df[key_predictors]

In [434]:
clf = DecisionTreeClassifier()
mlp = MLPClassifier()

In [435]:
def split_data(df):
    return df[df['confID'] == 0], df[df['confID'] == 1]

In [436]:


def train_model(year, model, data, label):


        train_data = data[data["year_x"] < year ]
        test_data  = data[data["year_x"] == year]

        X_train, X_test = train_data.drop([label], axis=1), test_data.drop([label], axis=1)
        y_train, y_test = train_data[label], test_data[label]

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        return y_test, y_pred
        


def train_evaluate_decision_tree_graph(model, data):

    accuracy_scores = []
    years_tested    = []

    data            = data.sort_values(by="year_x")
    data1, data2    = df[df['confID'] == 0], df[df['confID'] == 1]


    data1.drop(['confID'], axis=1, inplace=True)
    data2.drop(['confID'], axis=1, inplace=True)
    years           = sorted(data["year_x"].unique())

    for year in years[2:]:

        y_test1, y_pred1 = train_model(year, model, data1, label)
        y_test2, y_pred2 = train_model(year, model, data2, label)

        y_test = np.concatenate([y_test1, y_test2])
        y_pred = np.concatenate([y_pred1, y_pred2])

        accuracy = accuracy_score(y_test, y_pred)
        accuracy_scores.append(accuracy)
        years_tested.append(year)

    return years_tested, accuracy_scores

train_evaluate_decision_tree_graph(mlp, df)



/var/folders/97/nk3s9sxx36x77jsk9m_r0xmc0000gn/T/ipykernel_18532/2527847843.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1.drop(['confID'], axis=1, inplace=True)
/var/folders/97/nk3s9sxx36x77jsk9m_r0xmc0000gn/T/ipykernel_18532/2527847843.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2.drop(['confID'], axis=1, inplace=True)
/opt/homebrew/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


([2, 3, 4, 5, 6, 7, 8, 9],
 [0.8137931034482758,
  0.8282442748091603,
  0.922509225092251,
  0.9596774193548387,
  0.9194915254237288,
  0.907563025210084,
  0.9817518248175182,
  0.9651162790697675])